# Datathon data-extraction

## Parameters

In [1]:
BASE_DIR = '/Users/efraflores/Desktop/EF/Contests/Datathon_202109/data'
ZIP_NAME = 'smart_destinations_v2.zip'

## Unzip

In [2]:
import zipfile
from pathlib import Path

BASE_DIR = Path(BASE_DIR)

with zipfile.ZipFile(BASE_DIR.joinpath(ZIP_NAME), 'r') as zip_ref:
    zip_ref.extractall(BASE_DIR)

In [3]:
list(BASE_DIR.glob('*'))

[PosixPath('/Users/efraflores/Desktop/EF/Contests/Datathon_202109/data/datos'),
 PosixPath('/Users/efraflores/Desktop/EF/Contests/Datathon_202109/data/.DS_Store'),
 PosixPath('/Users/efraflores/Desktop/EF/Contests/Datathon_202109/data/ocean_elements.zip'),
 PosixPath('/Users/efraflores/Desktop/EF/Contests/Datathon_202109/data/ocean_fondos.zip'),
 PosixPath('/Users/efraflores/Desktop/EF/Contests/Datathon_202109/data/smart_destinations_v2.zip')]

In [4]:
import pandas as pd

aux = []
for x in BASE_DIR.glob('**/*.csv'):
    file_path = str(x)
    file_folder = file_path.split('/')[-2]
    file_name = file_path.split('/')[-1]
    sub_df = pd.read_csv(file_path, iterator=True)
    one_row = sub_df.get_chunk(1)
    cols = one_row.columns.tolist()
    aux.append((file_folder, file_name, len(cols), cols))

print(aux[:3])

[('datos', 'Finished_weather.csv', 482, ['comunity_code', 'valor_count_prec_1', 'valor_count_prec_10', 'valor_count_prec_11', 'valor_count_prec_12', 'valor_count_prec_2', 'valor_count_prec_3', 'valor_count_prec_4', 'valor_count_prec_5', 'valor_count_prec_6', 'valor_count_prec_7', 'valor_count_prec_8', 'valor_count_prec_9', 'valor_count_presMax_1', 'valor_count_presMax_10', 'valor_count_presMax_11', 'valor_count_presMax_12', 'valor_count_presMax_2', 'valor_count_presMax_3', 'valor_count_presMax_4', 'valor_count_presMax_5', 'valor_count_presMax_6', 'valor_count_presMax_7', 'valor_count_presMax_8', 'valor_count_presMax_9', 'valor_count_presMin_1', 'valor_count_presMin_10', 'valor_count_presMin_11', 'valor_count_presMin_12', 'valor_count_presMin_2', 'valor_count_presMin_3', 'valor_count_presMin_4', 'valor_count_presMin_5', 'valor_count_presMin_6', 'valor_count_presMin_7', 'valor_count_presMin_8', 'valor_count_presMin_9', 'valor_count_sol_1', 'valor_count_sol_10', 'valor_count_sol_11', 'val

In [5]:
df = pd.DataFrame(aux, columns=['folder','file_name','n_columns','columns'])
df['clean_columns'] = df['columns'].map(lambda x: ' '.join(x).replace('_',' '))
df[['columns','clean_columns']].sample(6)

,columns,clean_columns
163,"[zipcode, country, day, cards]",zipcode country day cards
86,"[zipcode, country, day, avg_amount]",zipcode country day avg amount
389,"[ccaa, indicador, anyo, valor]",ccaa indicador anyo valor
175,"[zipcode, country, day, cards]",zipcode country day cards
280,"[indicador, servicio, movimiento, cod_origen, ...",indicador servicio movimiento cod origen orige...
70,"[zipcode, country, day, cards]",zipcode country day cards


In [6]:
from sklearn.feature_extraction.text import CountVectorizer

cv = CountVectorizer()

X = cv.fit_transform(df['clean_columns']).toarray()
X = pd.DataFrame(X, columns=cv.get_feature_names(), index=df['file_name'])
X.sample(6)

,000,10,11,12,15,16,18,25,29,30,...,vela,velmedia,venue,viajeros,vinos,visitantes,visitor,visitors,week,zipcode
file_name,,,,,,,,,,,,,,,,,,,,,
ADR_RevPAR_ingresos_hoteleras.csv,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
avg_amount_35006.csv,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
cards_35307.csv,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
avg_amount_35109.csv,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
avg_amount_35138.csv,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
cards_35013.csv,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


In [7]:
from sklearn.cluster import KMeans

km = KMeans(13)
X['cluster'] = km.fit_predict(X)
df = df.set_index('file_name').join(X[['cluster']]).sort_values('cluster')
df.sample()

,folder,n_columns,columns,clean_columns,cluster
file_name,,,,,
venues_2021-03-01.csv,CM01_1,5,"[day, venue, mentions, name, value]",day venue mentions name value,5


In [8]:
df.to_csv(BASE_DIR.parent.joinpath('all_files_columns.csv'), sep='\t', encoding='utf-16')